In [ ]:
import numpy as np
import pandas as pd
# import requests
# import json
# import os.path
# import time
from concurrent.futures import ThreadPoolExecutor
from pynytimes import NYTAPI
NYT_API_KEY='e1RpBU3JBdcJ0dWyz6mB61EiXWG1DsM8'

In [ ]:
nyt = NYTAPI(NYT_API_KEY, parse_dates=True)

In [ ]:
data = pd.read_csv('data/ai_concat_pivot.csv')
# data

In [ ]:
# ind = 8

# article = data.loc[ind, :]
# article_title = "dashdyuisgfkasdgf" #article['Title']
# url = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={article_title}&api-key={NYT_API_KEY}' # &fq=source:("The New York Times")
# query = requests.get(url)
# query_data = query.json()['response']['docs'][0]

# query_data

In [ ]:
# # Find the indices of the missing dates
# missing_dates_indices = data['Article Date'].isnull()

# # Extract the titles corresponding to the missing dates
# missing_titles = data.loc[missing_dates_indices, 'Title']

# # Function to fetch article dates using the New York Times API
# def fetch_article_dates(titles):
#     titles_len = len(titles)
#     article_dates = []
#     for i, title in enumerate(titles):
#         try:
#             url = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={title}&api-key={NYT_API_KEY}' # &fq=source:("The New York Times")
#             # article = nyt.article_search(query=title, results=1)
#             query = requests.get(url)
#             query_data = query.json()['response']['docs'][0]
#             article_date = query_data['pub_date']
#             # Convert to the desired format
#             article_date = article_date.strftime("%Y-%m-%d %H:%M:%S")
#             article_dates.append(article_date)
#             print(f"Progress: {i}/{titles_len}")
#         except Exception as e:
#             print("Error: {}".format(e))
#             article_dates.append(np.nan)
#     return article_dates

# # Batch processing - fetch the article dates for the missing titles
# article_dates = fetch_article_dates(missing_titles)

# # Update the DataFrame with the fetched article dates
# data.loc[missing_dates_indices, 'Article Date'] = article_dates

# # Print progress
# print(f"Progress: {len(missing_titles)}/{len(data)}")

In [ ]:
# # Check if progress file exists
# progress_file = 'progress.csv'
# if os.path.isfile(progress_file):
#     # If progress file exists, read the progress from it
#     progress = pd.read_csv(progress_file, index_col=0)
#     start_index = progress.index[-1] + 1
#     print(f"Resuming from index {start_index}")
# else:
#     # If progress file does not exist, start from the beginning
#     start_index = 0
#     progress = pd.DataFrame(columns=['Title', 'Article Date'])

# # Find the indices of the missing dates
# missing_dates_indices = data['Article Date'].isnull()

# # Extract the titles corresponding to the missing dates
# missing_titles = data.loc[missing_dates_indices, 'Title']

# # Function to fetch article dates using the New York Times API
# def fetch_article_dates(titles):
#     titles_len = len(titles)
#     article_dates = []
#     for i, title in enumerate(titles):
#         try:
#             article = nyt.article_search(query=title, results=1)
#             if article:
#                 article_date = article[0]['pub_date']
#                 # Convert to the desired format
#                 article_date = article_date.strftime("%Y-%m-%d %H:%M:%S")
#                 article_dates.append(article_date)
#             else:
#                 print(f"Error: No article found for {title}")
#                 article_dates.append(np.nan)
#             print(f"Progress: {i}/{titles_len}")
#         except Exception as e:
#             print("Error: {}".format(e))
#             article_dates.append(np.nan)
#         # Write progress to CSV file after every 10 titles
#         if i % 10 == 0:
#             progress.loc[start_index+i] = [title, article_dates[-1]]
#             progress.to_csv(progress_file)
#             time.sleep(1) # Wait for 1 second to avoid API rate limit

#     return article_dates

In [ ]:
# Batch processing - fetch the article dates for the missing titles
article_dates = fetch_article_dates(missing_titles[start_index:])

# Update the DataFrame with the fetched article dates
data.loc[missing_dates_indices, 'Article Date'] = article_dates

# Print progress
print(f"Progress: {len(missing_titles)}/{len(data)}")

# Write progress to CSV file
progress.loc[start_index:start_index+len(article_dates)-1] = [missing_titles[start_index:], article_dates]
progress.to_csv(progress_file)

In [8]:
# Find the indices of the missing dates
missing_dates_indices = data['Article Date'].isnull()

# Extract the titles corresponding to the missing dates
missing_titles = data.loc[missing_dates_indices, 'Title']

print(len(missing_titles))

# Function to fetch article dates using the New York Times API
def fetch_article_dates(titles):
    titles_len = len(titles)
    article_dates = []
    for i, title in enumerate(titles):
        try:
            article = nyt.article_search(query=title, results=1)
            if article:
                article_date = article[0]['pub_date']
                # Convert to the desired format
                article_date = article_date.strftime("%Y-%m-%d %H:%M:%S")
                article_dates.append(article_date)
            else:
                print(f"Error: No article found for {title}")
                article_dates.append(np.nan)
            print(f"Progress: {i}/{titles_len}")
        except Exception as e:
            print("Error: {}".format(e))
            article_dates.append(np.nan)
    return article_dates

# Batch processing - fetch the article dates for the missing titles
# article_dates = fetch_article_dates(missing_titles)

# Update the DataFrame with the fetched article dates
# data.loc[missing_dates_indices, 'Article Date'] = article_dates

1489


In [9]:
# Try with parallel processing
with ThreadPoolExecutor() as executor:
    article_dates = list(executor.map(fetch_article_dates, missing_titles))

data.loc[missing_dates_indices, 'Article Date'] = article_dates

In [ ]:
# use pynytimes library to fill article dates of missing values in 'Article Date' column of dataset
for i in range(len(data)):
    if pd.isnull(data['Article Date'][i]):
        try:
            article = nyt.article_search(
                query = data['Title'][i],
                results = 1,
            )
            # get missing article date
            article_date = article[0]['pub_date']
            # convert article_date datetime.datetime (e.g: 1993-09-22 05:00:00+00:00) to yyyy-mm-dd hh:mm:ss date format
            article_date = article_date.strftime("%Y-%m-%d %H:%M:%S") 
            # NOTE: This is inefficient
            data.loc[i, 'Article Date'] = article_date
            # print progress
            print(f"Progress: {i}/{len(data)}")
        except Exception as e:
            print("Error: {}".format(e))
            # if there is an error, set article date to NaN
            data.loc[i, 'Article Date'] = np.nan
            pass

In [ ]:
data

In [ ]:
# Count NaNs in 'Article Date' column
print(f"Missing values in 'Article Date' column: {data['Article Date'].isnull().sum()} / {data.shape[0]} ({round(data['Article Date'].isnull().sum()/data.shape[0]*100, 2)}%)")